In [ ]:
!python -m spacy download en_core_web_lg

In [90]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion, Pipeline
import re
import spacy
from spacy.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from xgboost import XGBRegressor
from scipy.sparse import hstack, csr_matrix
from sklearn.feature_extraction import DictVectorizer
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as MSE

## EDA, Cleaning, Train Test Split

In [47]:
df = pd.read_csv('train.csv')
df.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,NYC,"Beautiful, sunlit brownstone 1-bedroom in the ...",2016-06-18,t,t,NaN,2012-03-26,f,2016-07-18,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,NYC,Enjoy travelling during your stay in Manhattan...,2017-08-05,t,f,100%,2017-06-19,t,2017-09-23,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,NYC,The Oasis comes complete with a full backyard ...,2017-04-30,t,t,100%,2016-10-25,t,2017-09-14,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,SF,This light-filled home-away-from-home is super...,NaN,t,t,NaN,2015-04-19,f,NaN,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,DC,"Cool, cozy, and comfortable studio located in ...",2015-05-12,t,t,100%,2015-03-01,t,2017-01-22,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0


In [48]:
df = df.drop(['bed_type', 'city', 'first_review', 'host_has_profile_pic', 'host_identity_verified', 
              'host_response_rate', 'last_review', 'neighbourhood', 'thumbnail_url', 'host_since', 'id', 'latitude', 
              'longitude', 'name', 'number_of_reviews', 'review_scores_rating', 'amenities', 'instant_bookable', 
              'cleaning_fee', 'zipcode'], axis=1)
print(df.shape)
df.head()

(74111, 9)


,log_price,property_type,room_type,accommodates,bathrooms,cancellation_policy,description,bedrooms,beds
0,5.010635,Apartment,Entire home/apt,3,1.0,strict,"Beautiful, sunlit brownstone 1-bedroom in the ...",1.0,1.0
1,5.129899,Apartment,Entire home/apt,7,1.0,strict,Enjoy travelling during your stay in Manhattan...,3.0,3.0
2,4.976734,Apartment,Entire home/apt,5,1.0,moderate,The Oasis comes complete with a full backyard ...,1.0,3.0
3,6.620073,House,Entire home/apt,4,1.0,flexible,This light-filled home-away-from-home is super...,2.0,2.0
4,4.744932,Apartment,Entire home/apt,2,1.0,moderate,"Cool, cozy, and comfortable studio located in ...",0.0,1.0


In [49]:
df.dropna(inplace=True)

In [50]:
df.shape

(73766, 9)

In [51]:
X = df.drop('log_price', axis=1)
y = df.log_price

In [52]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2, random_state=23)

## Transforming text and data into a sparse matrix so it can be analyzed with model

In [53]:
X_train = X_train.copy()
X_train['description'] = X_train['description'].str.lower()

X_val = X_val.copy()
X_val['description'] = X_val['description'].str.lower()

In [54]:
X_train['description'] = X_train['description'].replace('[^a-zA-Z0-9]', ' ', regex = True)
X_val['description'] = X_val['description'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [55]:
tf = TfidfVectorizer()

In [56]:
## Make sure to only transform validation/testing data as the transformer has already been fit
X_train_tfidf = tf.fit_transform(X_train['description'])
X_val_tfidf = tf.transform(X_val['description'])

In [57]:
enc = DictVectorizer()

In [100]:
X_train_tfidf

<59012x44707 sparse matrix of type '<class 'numpy.float64'>'
	with 5045998 stored elements in Compressed Sparse Row format>

In [97]:
X_train_categ.toarray()

array([[7. , 2.5, 3. , ..., 1. , 0. , 0. ],
       [2. , 1. , 1. , ..., 0. , 1. , 0. ],
       [5. , 2. , 2. , ..., 1. , 0. , 0. ],
       ...,
       [3. , 1. , 1. , ..., 1. , 0. , 0. ],
       [4. , 2. , 2. , ..., 1. , 0. , 0. ],
       [6. , 1. , 2. , ..., 1. , 0. , 0. ]])

In [58]:
## Make sure to only transform validation/testing data as the transformer has already been fit
X_train_categ = enc.fit_transform(X_train[['property_type', 'room_type', 'accommodates', 'bathrooms', 'cancellation_policy', 'bedrooms', 'beds']].to_dict('records'))
X_val_categ = enc.transform(X_val[['property_type', 'room_type', 'accommodates', 'bathrooms', 'cancellation_policy', 'bedrooms', 'beds']].to_dict('records'))

In [59]:
# Using hstack to concat the 2 arrays
X_trained = hstack([X_train_tfidf,X_train_categ])
X_vals = hstack([X_val_tfidf, X_val_categ])

In [60]:
# Using ridge regression model
clf = Ridge(alpha=1.0, random_state=23)

In [61]:
clf.fit(X_trained, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=23, solver='auto', tol=0.001)

In [62]:
rslt = clf.predict(X_vals)

In [63]:
rslt

array([4.9810281 , 5.74680193, 5.40458931, ..., 4.89828397, 4.16826015,
       4.29090455])

In [88]:
# Adding predictions of validation data to DF and transforming price from log price to a dollar amount
X_val['pred_price'] = rslt
X_val['pred_price'] = np.exp(X_val['pred_price'])
X_val['actual_price'] = y_val
X_val['actual_price'] = np.exp(X_val['actual_price'])
X_val.head()

,property_type,room_type,accommodates,bathrooms,cancellation_policy,description,bedrooms,beds,pred_price,actual_price
19856,Apartment,Entire home/apt,3,1.0,flexible,a spacious 1 bedroom garden level apartment r...,1.0,2.0,145.624021,130.0
35907,Apartment,Entire home/apt,4,1.0,moderate,this one bedroom apartment is central to all o...,1.0,1.0,313.187461,200.0
13857,Apartment,Entire home/apt,6,2.0,strict,newly renovated 2 bed 2 bath hollywood apartm...,2.0,2.0,222.424853,200.0
46774,Apartment,Entire home/apt,8,1.0,strict,my place is close to central park you ll love...,3.0,3.0,299.332459,425.0
8711,Apartment,Private room,1,1.0,strict,enjoy your fall in the amazing neighborhood of...,1.0,1.0,73.918323,51.0


In [92]:
MSE(rslt,y_val)

0.17900736767896405

In [95]:
clf.score(X_vals, y_val)

0.6516807847980268

In [98]:
X_val.head(15)

,property_type,room_type,accommodates,bathrooms,cancellation_policy,description,bedrooms,beds,pred_price,actual_price
19856,Apartment,Entire home/apt,3,1.0,flexible,a spacious 1 bedroom garden level apartment r...,1.0,2.0,145.624021,130.0
35907,Apartment,Entire home/apt,4,1.0,moderate,this one bedroom apartment is central to all o...,1.0,1.0,313.187461,200.0
13857,Apartment,Entire home/apt,6,2.0,strict,newly renovated 2 bed 2 bath hollywood apartm...,2.0,2.0,222.424853,200.0
46774,Apartment,Entire home/apt,8,1.0,strict,my place is close to central park you ll love...,3.0,3.0,299.332459,425.0
8711,Apartment,Private room,1,1.0,strict,enjoy your fall in the amazing neighborhood of...,1.0,1.0,73.918323,51.0
47578,House,Private room,1,1.0,flexible,this is a quiet back bedroom of a three bedroo...,1.0,1.0,73.108054,99.0
41123,Loft,Entire home/apt,5,1.0,strict,sunny giant loft 2 blocks from l train morgan ...,3.0,3.0,194.363857,250.0
13704,Apartment,Private room,2,1.0,strict,experience the city like a real new yorker aw...,1.0,1.0,76.068898,105.0
9099,Apartment,Entire home/apt,7,2.0,strict,d but du message r exp di kid an family f...,2.0,5.0,231.553334,350.0
22527,Apartment,Entire home/apt,4,1.0,moderate,modern studio across the street from santa mon...,0.0,2.0,181.505487,129.0


In [99]:
X_val.tail(15)

,property_type,room_type,accommodates,bathrooms,cancellation_policy,description,bedrooms,beds,pred_price,actual_price
145,Apartment,Private room,2,1.0,strict,1 bdrm avail in 2 bdrm apt bedroom with the h...,1.0,1.0,125.002921,100.0
54404,Apartment,Entire home/apt,2,1.0,flexible,located on the edge of glover archbald park e...,1.0,1.0,122.132167,300.0
32438,Apartment,Entire home/apt,4,1.0,strict,brand new 950 sqft ultra luxury designer sky s...,1.0,3.0,184.007114,211.0
69654,Townhouse,Entire home/apt,4,2.0,flexible,beautiful modern 2 br 2 br townhouse in jp wi...,2.0,2.0,187.136862,140.0
35292,House,Private room,2,1.5,strict,cozyb1 br in a clean comfortable and safe env...,1.0,1.0,71.500630,55.0
6792,Apartment,Private room,2,2.0,strict,beautiful retreat with stunning views in the h...,1.0,1.0,125.688164,100.0
33750,House,Entire home/apt,8,1.0,strict,our beautiful melrose home is located just off...,2.0,4.0,214.530148,175.0
72006,Bungalow,Entire home/apt,3,1.0,strict,private hollywood hills house of a fashion sty...,0.0,1.0,167.781329,108.0
12701,Condominium,Private room,2,1.0,moderate,my place is in rogersedge close to loyola un...,1.0,1.0,37.544348,35.0
36905,Condominium,Private room,2,1.0,strict,my condo is walking distance from coffee shops...,1.0,1.0,107.877051,80.0


## Testing different ideas until I figured out one that worked which is above

In [8]:
nlp = spacy.load("en_core_web_lg")

In [9]:
def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [57]:
tf = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=1,
                     max_df=.99,
                     stop_words='english',
                     tokenizer=tokenize)


In [58]:
tfidf_matrix = tf.fit_transform(df['description'])

In [59]:
tfidf_matrix.shape

(73766, 1088069)

In [12]:
dtm = pd.DataFrame(tfidf_matrix.todense(), columns=tf.get_feature_names())

In [13]:
dtm.tail()

,,2,apartment,bedroom,close,great,kitchen,locate,private,room,walk,$,+,1,1 bedroom,1 block,10,10 min,10 minute,12,15,15 min,15 minute,2,2 bathroom,2 bedroom,2 block,20,20 min,20 minute,24,24 hour,24/7,25,2nd,3,3 bedroom,3 block,30,3rd,...,vintage,visit,w,w/,walk,walk away,walk distance,walking,walking distance,wall,want,warm,washer,washer dryer,washington,watch,water,way,week,weekend,welcome,west,white,wi,wi fi,wifi,williamsburg,window,wine,wireless,wireless internet,wonderful,wood,work,world,yard,year,york,york city,young
73761,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
73762,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.23072,0.0,0.19963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
73763,0.046409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.091483,0.0,0.115982,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.247779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
73764,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.134963,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.232946,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.189793,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
73765,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.15798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.141386,0.126768,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.084896,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.122883,0.0,0.0,0.0,0.0,0.0


In [14]:
df.reset_index(inplace=True)

In [15]:
df.drop('index', axis=1)

,log_price,property_type,room_type,accommodates,bathrooms,cancellation_policy,description,bedrooms,beds
0,5.010635,Apartment,Entire home/apt,3,1.0,strict,"Beautiful, sunlit brownstone 1-bedroom in the ...",1.0,1.0
1,5.129899,Apartment,Entire home/apt,7,1.0,strict,Enjoy travelling during your stay in Manhattan...,3.0,3.0
2,4.976734,Apartment,Entire home/apt,5,1.0,moderate,The Oasis comes complete with a full backyard ...,1.0,3.0
3,6.620073,House,Entire home/apt,4,1.0,flexible,This light-filled home-away-from-home is super...,2.0,2.0
4,4.744932,Apartment,Entire home/apt,2,1.0,moderate,"Cool, cozy, and comfortable studio located in ...",0.0,1.0
...,...,...,...,...,...,...,...,...,...
73761,4.605170,Apartment,Private room,1,1.0,flexible,one room in bushwick apt,1.0,1.0
73762,5.043425,Apartment,Entire home/apt,4,2.0,moderate,"Located on the Pacific Coast Highway, this apa...",2.0,4.0
73763,5.220356,Apartment,Entire home/apt,5,1.0,moderate,A modern apartment located in East Williamsbur...,2.0,2.0
73764,5.273000,Apartment,Entire home/apt,2,1.0,strict,"Fully renovated, designer's studio located in ...",0.0,2.0


In [16]:
def reduceCardinality(series):
  top10 = series.value_counts()[:10].index
  return series.apply(lambda x: 'Other' if x not in top10 else x)

In [17]:
df['property_type'] = reduceCardinality(df['property_type'])

In [18]:
df['property_type'].value_counts()

Apartment          48772
House              16432
Condominium         2654
Townhouse           1690
Other               1481
Loft                1239
Guesthouse           497
Bed & Breakfast      458
Bungalow             365
Villa                178
Name: property_type, dtype: int64

In [19]:
new_df = pd.concat([df, dtm], axis=1)

In [20]:
new_df = new_df.copy()
X = new_df.drop(['log_price', 'description', 'index'], axis=1)
y = new_df.log_price

In [21]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2, random_state=23)

In [22]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(59012, 754)
(14754, 754)
(59012,)
(14754,)


In [36]:
clf = Ridge()
ord = OrdinalEncoder()
ohe = OneHotEncoder()

In [23]:
X_train.head()

,property_type,room_type,accommodates,bathrooms,cancellation_policy,bedrooms,beds,,2,apartment,bedroom,close,great,kitchen,locate,private,room,walk,$,+,1,1 bedroom,1 block,10,10 min,10 minute,12,15,15 min,15 minute,2,2 bathroom,2 bedroom,2 block,20,20 min,20 minute,24,24 hour,24/7,...,vintage,visit,w,w/,walk,walk away,walk distance,walking,walking distance,wall,want,warm,washer,washer dryer,washington,watch,water,way,week,weekend,welcome,west,white,wi,wi fi,wifi,williamsburg,window,wine,wireless,wireless internet,wonderful,wood,work,world,yard,year,york,york city,young
59505,Apartment,Entire home/apt,7,2.5,flexible,3.0,3.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
66257,Apartment,Private room,2,1.0,moderate,1.0,1.0,0.053903,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.113354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.115943,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
65156,Apartment,Entire home/apt,5,2.0,strict,2.0,2.0,0.245467,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.083301,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.077616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.031632,0.0,0.0,0.082342,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.076427,0.077689,0.050996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.059594,0.073814,0.0,0.0,0.0,0.0,0.0
57242,Apartment,Private room,2,1.0,strict,1.0,1.0,0.200736,0.0,0.0,0.0,0.0,0.128177,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.046562,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.075065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
64933,Apartment,Entire home/apt,2,1.0,strict,0.0,1.0,0.091414,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.096943,0.0,0.000000,0.0,0.0,0.05033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.14372,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [26]:
pipe = Pipeline([
                ('encode',OneHotEncoder()),
                #('impute', SimpleImputer()),
                #('scale',MinMaxScaler()),
                ('model',Ridge(random_state=41))
                ])
pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('encode',
                 OneHotEncoder(categories='auto', drop=None,
                               dtype=<class 'numpy.float64'>,
                               handle_unknown='error', sparse=True)),
                ('model',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=41,
                       solver='auto', tol=0.001))],
         verbose=False)

In [48]:
y_pred = pipe.predict(X_val)

ValueError: ignored

In [49]:
model = Ridge(solver="sag", fit_intercept=True, random_state=369)

In [50]:
pipe = Pipeline([
                ('encode',OneHotEncoder()),
                #('impute', SimpleImputer()),
                #('scale',MinMaxScaler()),
                ('model',model)
                ])

In [51]:
pipe.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:558: UserWarning: "sag" solver requires many iterations to fit an intercept with sparse inputs. Either set the solver to "auto" or "sparse_cg", or set a low "tol" and a high "max_iter" (especially if inputs are not standardized).
  '"sag" solver requires many iterations to fit '


Pipeline(memory=None,
         steps=[('encode',
                 OneHotEncoder(categories='auto', drop=None,
                               dtype=<class 'numpy.float64'>,
                               handle_unknown='error', sparse=True)),
                ('model',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=369,
                       solver='sag', tol=0.001))],
         verbose=False)

In [52]:
y_pred = pipe.predict(X_val)

ValueError: ignored

In [70]:
X_dummies = csr_matrix(pd.get_dummies(df[['property_type', 'room_type', 'cancellation_policy']], sparse=True).values)

In [69]:
df.dtypes

index                    int64
log_price              float64
property_type           object
room_type               object
accommodates             int64
bathrooms              float64
cancellation_policy     object
description             object
bedrooms               float64
beds                   float64
dtype: object

In [56]:
X_dummies

<73766x18 sparse matrix of type '<class 'numpy.uint8'>'
	with 221298 stored elements in Compressed Sparse Row format>

In [72]:
csr_df = hstack((X_dummies, tfidf_matrix)).tocsr()

In [73]:
X1 = csr_df

In [74]:
X1

<73766x1088087 sparse matrix of type '<class 'numpy.float64'>'
	with 10140249 stored elements in Compressed Sparse Row format>

In [78]:
model = Ridge(solver="saga", fit_intercept=False, random_state=23)

In [81]:
model.fit(X1, y)

Ridge(alpha=1.0, copy_X=True, fit_intercept=False, max_iter=None,
      normalize=False, random_state=23, solver='saga', tol=0.001)

In [84]:
X1_train, X1_val, y1_train, y1_val = train_test_split(X1, y, test_size=.2, random_state=23)

In [89]:
X1_train.reshape(-1,1)

<64210190044x1 sparse matrix of type '<class 'numpy.float64'>'
	with 8117697 stored elements in COOrdinate format>

In [90]:
model.fit(X1_train, y1_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=False, max_iter=None,
      normalize=False, random_state=23, solver='saga', tol=0.001)

In [95]:
y_pred = model.predict(X_val)

ValueError: ignored

In [ ]:
x2 = pd.DataFrame(X1.todense())

In [98]:
x2.head()

,0
0,"(0, 0)\t1.0\n (0, 10)\t1.0\n (0, 15)\t1.0\..."
1,"(0, 0)\t1.0\n (0, 10)\t1.0\n (0, 15)\t1.0\..."
2,"(0, 0)\t1.0\n (0, 10)\t1.0\n (0, 14)\t1.0\..."
3,"(0, 5)\t1.0\n (0, 10)\t1.0\n (0, 13)\t1.0\..."
4,"(0, 0)\t1.0\n (0, 10)\t1.0\n (0, 14)\t1.0\..."
